# Uso de tracebacks para buscar errores

Las excepciones en Python son una característica principal del lenguaje. Es posible que te sorprenda leer que algo que genera errores se resalta como una característica. Esta sorpresa puede deberse a que las herramientas de software sólidas no parecen bloquearse con un traceback seguimiento (varias líneas de texto que indican cómo se inició y finalizó el error).

Sin embargo, las excepciones son útiles porque ayudan en la toma de decisiones generando mensajes de error descriptivos. Pueden ayudarte a controlar los problemas esperados e inesperados.

## Tracebacks

Un traceback es el cuerpo del texto que puede apuntar al origen (y al final) de un error no controlado. Comprender los componentes de un traceback hará que seas más eficaz al corregir errores o depurar un programa que no funciona bien.

La primera vez que encuentres excepciones en Python podrías verte tentado/a a evitar el error suprimiéndolo. Cuando un programa sufre un error no controlado, aparece un traceback como salida. Como verás en este módulo, los tracebacks son útiles. Hay maneras de controlar correctamente los errores para que no aparezcan o muestren información útil.

In [1]:
def main():
    open("/path/to/mars.jpg")

if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/mars.jpg'

Los tracebacks casi siempre incluyen la información siguiente:

    Todas las rutas de acceso de archivo implicadas, para cada llamada a cada función.
    
    Los números de línea asociados a cada ruta de acceso de archivo.
    
    Los nombres de las funciones, métodos o clases implicados en la generación de una excepción.
    
    El nombre de la excepción que se ha producido.


## Controlando las excepciones

Cuando encuentres por primera vez excepciones que muestren tracebacks grandes como salida, es posible que te veas tentado/a a detectar todos los errores para evitar que esto suceda.

Si estás en una misión a Marte, ¿Qué podrías hacer si un texto del sistema de navegación dice 'Se ha producido un error'? Imagina que no hay ninguna más información ni contexto, simplemente una luz roja parpadeante con el texto del error. Como desarrollador, resulta útil ponerse al otro lado del programa: ¿Qué puede hacer un usuario cuando se produce un error?

### Try y Except de los bloques

Vamos a usar el ejemplo de navegador a fin de crear código que abra archivos de configuración para la misión de Marte. Los archivos de configuración pueden tener todo tipo de problemas, por lo que es fundamental notificarlos con precisión cuando se presenten. Sabemos que, si no existe un archivo o directorio, se genera FileNotFoundError. Si queremos controlar esa excepción, podemos hacerlo con un bloque try y except:

In [2]:
try:
    open('config.txt')
except FileNotFoundError:
    print("No podemos encontrar el archivo!")

Couldn't find the config.txt file!


Después de la palabra clave try, agregamos código que tenga la posibilidad de producir una excepción. A continuación, agregamos la palabra clave except junto con la posible excepción, seguida de cualquier código que deba ejecutarse cuando se produce esa condición. Puesto que config.txt no existe en el sistema, Python imprime que el archivo de configuración no está ahí. El bloque try y except, junto con un mensaje útil, evita un seguimiento y sigue informando al usuario sobre el problema.


In [4]:
def main():
    try:
        configuracion = open('config.txt')
    except FileNotFoundError:
        print("No podemos encontrar el archivo")


if __name__ == '__main__':
    main()

Couldn't find the config.txt file!


Aunque es común un archivo que no existe, no es el único error que podemos encontrar. Los permisos de archivo no válidos pueden impedir la lectura de un archivo, incluso si este existe.

In [9]:
def main():
    try:
        configuracion = open('config.txt')
    except Exception:
        print("No podemos encontrar el archivo")


if __name__ == '__main__':
    main()

Couldn't find the config.txt file!


El problema ahora es que el mensaje de error es incorrecto. El archivo existe, pero tiene permisos diferentes y Python no puede leerlo. Cuando se trata con errores de software, puede resultar frustrante tener errores que hagan lo siguiente:

    No indiquen cuál es el problema real.
    Proporcionen una salida que no coincida con el problema real.
    No sugieran lo que se puede hacer para corregir el problema.

Vamos a corregir este fragmento de código para abordar todas estas frustraciones. Revertiremos la detección de FileNotFoundError y luego agregamos otro bloque except para detectar PermissionError:

In [11]:
def main():
    try:
        configuration = open('config.txt')
    except FileNotFoundError:
        print("No podemos encontrar el archivo!")
    except IsADirectoryError:
        print("Encontramos el archivo, pero es un directorio y no podemos abrirlo")

if __name__ == '__main__':
    main()

Cuando los errores son de una naturaleza similar y no es necesario controlarlos individualmente, puedes agrupar las excepciones como una usando paréntesis en la línea except. Por ejemplo, si el sistema de navegación está bajo cargas pesadas y el sistema de archivos está demasiado ocupado, tiene sentido detectar BlockingIOError y TimeOutError juntos:

In [12]:
def main():
    try:
        configuracion = open('config.txt')
    except FileNotFoundError:
        print("No podemos encontrar el archivo!")
    except IsADirectoryError:
        print("Encontramos el archivo, pero es directorio y no podemos abrirlo")
    except (BlockingIOError, TimeoutError):
        print("Archivos de sistema bajo mucha demanda, no podemos completar la lectura del archivo")

if __name__ == '__main__':
    main()

Si necesitas acceder al error asociado a la excepción, debes actualizar la línea except para incluir la palabra clave as. Esta técnica es práctica si una excepción es demasiado genérica y el mensaje de error puede ser útil

En este caso, as err significa que err se convierte en una variable con el objeto de excepción como valor. Después, usa este valor para imprimir el mensaje de error asociado a la excepción. Otra razón para usar esta técnica es acceder directamente a los atributos del error. Por ejemplo, si detecta una excepción OSError más genérica, que es la excepción primaria de FilenotFoundError y PermissionError, podemos diferenciarlas mediante el atributo .errno.

In [13]:
try:
    open("config.txt")
except OSError as err:
    if err.errno == 2:
        print("No podemos encontrar el archivo!")
    elif err.errno == 13:
        print("Archivo encontrado pero no podemos leerlo")

Couldn't find the config.txt file!


## Generación de excepciones

Es posible que ya conozcas una situación que podría provocar una condición de error al escribir código. En estas situaciones, resulta útil generar excepciones que permitan que otro código comprenda cuál es el problema.

La generación de excepciones también puede ayudar en la toma de decisiones para otro código. Como hemos visto antes, en función del error, el código puede tomar decisiones inteligentes para resolver, solucionar o ignorar un problema.

Los astronautas limitan su uso de agua a unos 11 litros al día. Vamos a crear una función que, con base al número de astronautas, pueda calcular la cantidad de agua quedará después de un día o más:

In [19]:
def calculoAgua(astronatas, provisiones, diasFaltantes):
    for argument in [astronatas, provisiones, diasFaltantes]:
        try:
            argument / 10
        except TypeError:
            raise TypeError(f"All arguments must be of type int, but received: '{argument}'")
    usoDiario = astronatas * 11
    usoRestante = usoDiario * diasFaltantes
    totalAguaRestante = provisiones - usoRestante
    if totalAguaRestante < 0:
        raise RuntimeError(f"No hay suficiente agua para {astronatas} astronautas para {diasFaltantes} dias!")
    return f"Total de agua disponible para {diasFaltantes} dias: {totalAguaRestante} litros"

def alertaNavegantes(err):
    print(err)

try: 
    #calculoAgua("3", "200", None)
    calculoAgua(5, 100, 2)
except RuntimeError as err:
    alertaNavegantes(err)

No hay suficiente agua para 5 astronautas para 2 dias!
